In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import keras

from keras import Model
from keras import layers
from sklearn.model_selection import train_test_split
from PIL import Image
import random



In [2]:
# Paths
BASE_DIR = r'C:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\Capstone'
IMAGES_DIR = os.path.join(BASE_DIR, 'nutrition5k', 'images', 'theframes')
CSV_PATH = os.path.join(BASE_DIR, 'nutrition5k', 'metadata', '3_dishmetadata_weightonly_theframes_nosingular.csv')
INGREDIENT_METADATA = os.path.join(BASE_DIR, 'nutrition5k', 'metadata', '3_ingredientmetadata_theframes_cleaned.csv')



In [3]:
# Load the ingredient metadata
ingredient_metadata = pd.read_csv(INGREDIENT_METADATA)
ingredient_id_map = {row['id']: row['ingr'] for _, row in ingredient_metadata.iterrows()}

# Load dish metadata
dish_metadata = pd.read_csv(CSV_PATH, header=None)


In [4]:
def load_images(dish_id, image_dir, target_size=(224, 224)):
    """
    Load all images for a given dish_id, resize them, and return as a numpy array.
    Handles nested folder structure and inconsistent naming conventions for JPEGs.
    """
    # Path to the frames_sampled30 folder
    dish_folder = os.path.join(image_dir, str(dish_id), 'frames_sampled30')
    
    if not os.path.exists(dish_folder):
        print(f"Warning: Directory not found for dish ID {dish_id}. Skipping.")
        return []  # Return an empty list if the folder doesn't exist
    
    images = []
    for file in os.listdir(dish_folder):
        if file.lower().endswith('.jpeg'):  # Case-insensitive check for .jpeg files
            img_path = os.path.join(dish_folder, file)
            try:
                img = Image.open(img_path).convert('RGB').resize(target_size)
                images.append(np.array(img))
            except Exception as e:
                print(f"Error loading image {img_path}: {e}")
    return images

In [5]:
def load_images(dish_id, image_dir, target_size=(224, 224)):
    """
    Load all images for a given dish_id, resize them, and return as a numpy array.
    """
    dish_folder = os.path.join(image_dir, f'{dish_id}', 'frames_sampled30')
    images = []
    for file in os.listdir(dish_folder):
        if file.endswith('.jpeg'):
            img_path = os.path.join(dish_folder, file)
            img = Image.open(img_path).convert('RGB').resize(target_size)
            images.append(np.array(img))
    return np.array(images)

In [7]:
def split_dataset(dish_metadata, image_dir):
    # Lists to hold images and corresponding labels/weights
    train_images = []
    train_labels = []
    train_weights = []
    val_images = []
    val_labels = []
    val_weights = []
    test_images = []
    test_labels = []
    test_weights = []

    max_ingredients = 26  # Max ingredients per dish
    num_classes = 246  # Total possible ingredients

    for idx, (_, row) in enumerate(dish_metadata.iterrows()):
        dish_id = str(row[0])
        images = load_images(dish_id, image_dir)
        if images is None or len(images) == 0:
            continue  # Skip if no images were loaded
        
        images = np.array(images)
        np.random.shuffle(images)

        # Split the images into test, validation, and training
        test_image = images[0]  # First image for testing
        val_images_from_dish = images[1:3]  # Two random images for validation
        train_images_from_dish = images[3:]  # Remaining images for training

        # Add to corresponding sets
        test_images.append(test_image)
        val_images.extend(val_images_from_dish)
        train_images.extend(train_images_from_dish)

        # Prepare labels and weights for the ingredients in the dish
        ingredient_ids, weights = [], []
        for i in range(2, len(row), 3):
            if pd.isna(row[i]):
                break
            ing_id = int(row[i]) - 1  # Adjusting for zero-indexing
            if ing_id < num_classes:  # Ensure valid ingredient ID
                ingredient_ids.append(ing_id)
                weights.append(float(row[i + 2]))

        # Pad or truncate ingredient IDs and weights
        while len(ingredient_ids) < max_ingredients:
            ingredient_ids.append(0)  # Padding with 0
        ingredient_ids = ingredient_ids[:max_ingredients]

        while len(weights) < max_ingredients:
            weights.append(0.0)  # Padding with 0.0
        weights = weights[:max_ingredients]

        label = keras.utils.to_categorical(ingredient_ids, num_classes=num_classes)

        # Add labels and weights to training, validation, and test sets
        train_labels.extend([label] * len(train_images_from_dish))
        train_weights.extend([weights] * len(train_images_from_dish))
        val_labels.extend([label] * len(val_images_from_dish))
        val_weights.extend([weights] * len(val_images_from_dish))
        test_labels.append(label)
        test_weights.append(weights)

    # Convert to NumPy arrays
    X_train = np.array(train_images)
    Y_train_class = np.array(train_labels)
    Y_train_weight = np.array(train_weights)
    X_val = np.array(val_images)
    Y_val_class = np.array(val_labels)
    Y_val_weight = np.array(val_weights)
    X_test = np.array(test_images)
    Y_test_class = np.array(test_labels)
    Y_test_weight = np.array(test_weights)

    return X_train, Y_train_class, Y_train_weight, X_val, Y_val_class, Y_val_weight, X_test, Y_test_class, Y_test_weight


In [8]:
# Example usage
X_train, Y_train_class, Y_train_weight, X_val, Y_val_class, Y_val_weight, X_test, Y_test_class, Y_test_weight = split_dataset(dish_metadata, IMAGES_DIR)

print(f"Train set: X={X_train.shape}, Y_class={Y_train_class.shape}, Y_weight={Y_train_weight.shape}")
print(f"Validation set: X={X_val.shape}, Y_class={Y_val_class.shape}, Y_weight={Y_val_weight.shape}")
print(f"Test set: X={X_test.shape}, Y_class={Y_test_class.shape}, Y_weight={Y_test_weight.shape}")

Train set: X=(40627, 224, 224, 3), Y_class=(40627, 26, 246), Y_weight=(40627, 26)
Validation set: X=(9432, 224, 224, 3), Y_class=(9432, 26, 246), Y_weight=(9432, 26)
Test set: X=(4794, 224, 224, 3), Y_class=(4794, 26, 246), Y_weight=(4794, 26)


In [9]:
def normalize_in_batches(data, batch_size=1000):
    for start in range(0, len(data), batch_size):
        end = start + batch_size
        data[start:end] = data[start:end] / 255.0
    return data

X_train = normalize_in_batches(X_train)
X_val = normalize_in_batches(X_val)
X_test = normalize_in_batches(X_test)


In [10]:
import numpy as np

assert not np.any(np.isnan(X_train)), "NaN detected in X_train"
assert not np.any(np.isnan(Y_train_class)), "NaN detected in Y_train_class"
assert not np.any(np.isnan(Y_train_weight)), "NaN detected in Y_train_weight"


In [43]:
from keras import layers, models, optimizers, regularizers
import tensorflow as tf

def create_model(input_shape=(224, 224, 3), num_ingredients=26, num_classes=246):
    inputs = layers.Input(shape=input_shape)

    # Convolutional layers with regularization
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(1e-3))(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(1e-3))(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same',
                      kernel_regularizer=regularizers.l2(1e-3))(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.BatchNormalization()(x)

    # Fully connected layers with dropout
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-3))(x)
    x = layers.Dropout(0.5)(x)

    # Classification output
    class_logits = layers.Dense(num_ingredients * num_classes, activation=None)(x)
    class_logits = layers.Reshape((num_ingredients, num_classes))(class_logits)
    class_logits = layers.Lambda(lambda logits: tf.clip_by_value(logits, -10.0, 10.0))(class_logits)  # Clip logits
    class_output = layers.Softmax(axis=-1, name='class_output')(class_logits)  # Apply softmax explicitlyt

    # Regression output
    regression_output = layers.Dense(num_ingredients, activation='linear', name='regression_output')(x)

    # Define the model
    model = models.Model(inputs=inputs, outputs=[class_output, regression_output])
    return model

# Create the model
model = create_model()

# Define the optimizer with learning rate
optimizer = optimizers.Adam(learning_rate=1e-4)

# Compile the model
model.compile(
    optimizer=optimizer,
    loss={
        'class_output': keras.losses.CategoricalCrossentropy(from_logits=False),
        'regression_output': keras.losses.MeanSquaredError()
    },
    metrics={
        'class_output': [keras.metrics.TopKCategoricalAccuracy(k=1), keras.metrics.TopKCategoricalAccuracy(k=5)],
        'regression_output': 'mae'
    }
)

# Print the model summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 224, 224,  │        896 │ input_layer_6[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_18    │ (None, 112, 112,  │          0 │ conv2d_18[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        128 │ max_pooling2d_18… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 112, 112,  │     18,496 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_19    │ (None, 56, 56,    │          0 │ conv2d_19[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 56, 56,    │        256 │ max_pooling2d_19… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 56, 56,    │     73,856 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_20    │ (None, 28, 28,    │          0 │ conv2d_20[0][0]   │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        512 │ max_pooling2d_20… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 256)       │     33,024 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 256)       │          0 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 6396)      │  1,643,772 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_6 (Reshape) │ (None, 26, 246)   │          0 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_6 (Lambda)   │ (None, 26, 246)   │          0 │ reshape_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ class_output        │ (None, 26, 246)   │          0 │ lambda_6[0][0]    │
│ (Softmax)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ regression_output   │ (None, 26)        │      6,682 │ dropout_6[0][0]   │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 1,777,622 (6.78 MB)

 Trainable params: 1,777,174 (6.78 MB)

 Non-trainable params: 448 (1.75 KB)

In [44]:
history = model.fit(
    [X_train], 
    {'class_output': Y_train_class, 'regression_output': Y_train_weight},
    validation_data=(
        [X_val], 
        {'class_output': Y_val_class, 'regression_output': Y_val_weight}
    ),
    epochs=10,
    batch_size=32,
    verbose=1
)


Epoch 1/10


c:\Users\Rafi Kyandra\VSCODE\Semester 5\Bangkit MSIB\Capstone\capstonevenv\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: keras_tensor_114. Received: the structure of inputs=('*',)
  warnings.warn(


1270/1270 ━━━━━━━━━━━━━━━━━━━━ 877s 686ms/step - class_output_loss: 3.3095 - class_output_top_k_categorical_accuracy: 0.7309 - class_output_top_k_categorical_accuracy_1: 0.7836 - loss: 5434.8867 - regression_output_loss: 5431.2598 - regression_output_mae: 6.5502 - val_class_output_loss: 1.6063 - val_class_output_top_k_categorical_accuracy: 0.8408 - val_class_output_top_k_categorical_accuracy_1: 0.8681 - val_loss: 2840.7947 - val_regression_output_loss: 2836.6934 - val_regression_output_mae: 7.4705
Epoch 2/10
1270/1270 ━━━━━━━━━━━━━━━━━━━━ 1139s 897ms/step - class_output_loss: 1.2444 - class_output_top_k_categorical_accuracy: 0.8587 - class_output_top_k_categorical_accuracy_1: 0.8856 - loss: 4641.2217 - regression_output_loss: 4639.5850 - regression_output_mae: 7.2473 - val_class_output_loss: 1.3582 - val_class_output_top_k_categorical_accuracy: 0.8409 - val_class_output_top_k_categorical_accuracy_1: 0.8673 - val_loss: 2836.0911 - val_regression_output_loss: 2832.1980 - val_regression_o

In [46]:
# Evaluate the model on the test set
test_results = model.evaluate(
    x=X_test,
    y={'class_output': Y_test_class, 'regression_output': Y_test_weight},
    batch_size=32
)

print("Test Results:", test_results)

150/150 ━━━━━━━━━━━━━━━━━━━━ 24s 156ms/step - class_output_loss: 1.1517 - class_output_top_k_categorical_accuracy: 0.8386 - class_output_top_k_categorical_accuracy_1: 0.8665 - loss: 2551.1768 - regression_output_loss: 2549.4946 - regression_output_mae: 7.4565
Test Results: [2772.745361328125, 1.1174829006195068, 2768.17578125, 0.8426077961921692, 0.8698375225067139, 7.4302568435668945]


In [23]:
print(model.output_names)


ListWrapper(['reshape', 'regression_output'])


In [47]:
# Save the trained model
model.save("ingredient_weight_model_1.h5")
print("Model saved as 'ingredient_weight_model_1.h5'")

Model saved as 'ingredient_weight_model_1.h5'


In [48]:
# Save the model in the new Keras format
model.save("ingredient_weight_model_1.keras")
print("Model saved as 'ingredient_weight_model_1.keras'")

Model saved as 'ingredient_weight_model_1.keras'
